##Import required library

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Venomhere\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##Read dataset and do pre-processing

In [2]:
data = pd.read_csv(r"C:\Users\Venomhere\Downloads\spam.csv",encoding='ISO-8859-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
data.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [4]:
data.describe()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2


In [5]:
data['alpha_text'] = data['v2'].apply(lambda x: re.sub(r'[^a-zA-Z ]+', '', x.lower()))
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,alpha_text
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,free entry in a wkly comp to win fa cup final...
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,nah i dont think he goes to usf he lives aroun...


In [6]:
data.shape

(5572, 6)

##NLP

In [7]:
ps=PorterStemmer()
message=[]
for i in range(0,5572):
  msg=data['v2'][i]
  msg=re.sub('[^a-zA-Z]',' ',msg)
  msg=msg.lower()
  msg=msg.split(' ')
  msg = [ps.stem(word) for word in msg if word not in set(stopwords.words('english'))]
  msg=' '.join(msg)
  message.append(msg)

message[:6]

['go jurong point  crazi   avail bugi n great world la e buffet    cine got amor wat   ',
 'ok lar    joke wif u oni   ',
 'free entri   wkli comp win fa cup final tkt   st may       text fa       receiv entri question std txt rate c appli             ',
 'u dun say earli hor    u c alreadi say   ',
 'nah think goe usf  live around though',
 'freemsg hey darl   week word back  like fun still  tb ok  xxx std chg send         rcv']

##Add Layers (LSTM, Dense-(Hidden Layers), Output

In [8]:
cv = CountVectorizer()
x = cv.fit_transform(message).toarray()
le = LabelEncoder()
data['v1']=le.fit_transform(data['v1'])
y = data['v1'].values

##Compile the Model



In [9]:
model = Sequential()
model.add(Dense(1550,activation='relu'))
model.add(Dense(3000,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(x,y,epochs=10)

Epoch 1/10
175/175 [==============================] - 17s 87ms/step - loss: 0.0989 - accuracy: 0.9733
Epoch 2/10
175/175 [==============================] - 15s 85ms/step - loss: 0.0099 - accuracy: 0.9977
Epoch 3/10
175/175 [==============================] - 15s 85ms/step - loss: 0.0025 - accuracy: 0.9993
Epoch 4/10
175/175 [==============================] - 15s 85ms/step - loss: 2.5458e-04 - accuracy: 1.0000
Epoch 5/10
175/175 [==============================] - 15s 86ms/step - loss: 8.9455e-05 - accuracy: 1.0000
Epoch 6/10
175/175 [==============================] - 15s 85ms/step - loss: 5.0223e-05 - accuracy: 1.0000
Epoch 7/10
175/175 [==============================] - 15s 87ms/step - loss: 3.1472e-05 - accuracy: 1.0000
Epoch 8/10
175/175 [==============================] - 16s 90ms/step - loss: 2.0957e-05 - accuracy: 1.0000
Epoch 9/10
175/175 [==============================] - 17s 95ms/step - loss: 1.4422e-05 - accuracy: 1.0000
Epoch 10/10
175/175 [==============================] - 15s

##Save The Model

In [10]:
model.save('spam_nlp.h5')

##TEST THE MODEL

In [11]:
# Function to check spam or not 

In [12]:
def check_spam(msg):
    msg=re.sub('[^a-zA-Z]',' ',msg)
    msg=msg.lower()
    msg=msg.split(' ')
    msg = [ps.stem(word) for word in msg if word not in set(stopwords.words('english'))]
    msg=' '.join(msg)
    predict = model.predict(cv.transform([msg]))
    if predict > 0.5:
        pred='Spam'
    else: 
        pred='Not Spam'
    return pred

In [13]:
message = "I HAVE A DATE ON SUNDAY WITH WILL!!"


In [18]:
print('The message is predicted as:',check_spam(message))

1/1 [==============================] - 0s 13ms/step
The message is predicted as: Not Spam
